In [1]:
#IMPORT NECESSARY LIBRARIES
from osgeo import gdal,ogr,osr
import os, sys
import folium
import geopandas as gpd

In [2]:
#IMPORT SHAPEFILES
#DEFINE THE DIRECTORY TO THE SHAPEFILE
inshpdir = input('INPUT VECTOR FILE: ')
#GET DRIVER BY NAME
driver = ogr.GetDriverByName('ESRI Shapefile')
#OPEN SHAPEFILE
dataset = driver.Open(inshpdir)
layer = dataset.GetLayer()

INPUT VECTOR FILE: D:\\Misc\\TESTVN2K\\Export_Output.shp


In [3]:
#DEFINE PROJECTED VN2000 (CAN SKIP IF THERE IS NO TRANSFORMATION FROM/TO VN2000)
#DEFINE CENTRAL MERIDIAN
centralmerid = input('PLEASE INPUT CENTRAL MERIDIAN OF INPUT FILE: ')
#DEFINE SCALE
scale = input('PLEASE CHOOSE SCALE OF INPUT FILE (0.9996/0.9999): ')
#DEFINE VN2000 PROJECTION
prj = '''PROJCS["VN-2000 {} /", 
    GEOGCS["VN-2000",_
        DATUM["Vietnam_2000",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            TOWGS84[-191.90441429,-39.30318279,-111.45032835,-0.00928836,0.01975479,-0.00427372,1.000000252906278],
            AUTHORITY["EPSG","6756"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4756"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",{}],
    PARAMETER["scale_factor",{}],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","3405"]]'''.format(centralmerid,centralmerid,scale)
vnspatref = osr.SpatialReference()
vnspatref.ImportFromWkt(prj)

PLEASE INPUT CENTRAL MERIDIAN OF INPUT FILE: 105.75
PLEASE CHOOSE SCALE OF INPUT FILE (0.9996/0.9999): 0.9999


0

In [4]:
#DEFINE GEOGRAPHIC VN2000 (CAN SKIP IF THERE IS NO TRANSFORMATION FROM/TO VN2000)
prj = '''GEOGCS["VN-2000",
    DATUM["Vietnam_2000",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        TOWGS84[-191.90441429,-39.30318279,-111.45032835,-0.00928836,0.01975479,-0.00427372,1.000000252906278],
        AUTHORITY["EPSG","6756"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4756"]]'''
geovnspatref = osr.SpatialReference()
geovnspatref.ImportFromWkt(prj)

0

In [5]:
#DEFINE UTM WGS84 (CAN SKIP IF THERE IS NO TRANSFORMATION FROM/TO UTM WGS84)
UTMcent = input('PLEASE CHOOSE UTM ZONE (48/49): ')
wgsspatref = osr.SpatialReference()
if int(UTMcent) == 48:
    wgsspatref.ImportFromEPSG(32648)
else:
    wgsspatref.ImportFromEPSG(32649)

PLEASE CHOOSE UTM ZONE (48/49): 48


In [6]:
#DEFINE GEOGRAPHIC WGS84 (CAN SKIP IF THERE IS NO TRANSFORMATION FROM/TO WGS84)
geowgsspatref = osr.SpatialReference()
geowgsspatref.ImportFromEPSG(4326)

0

In [7]:
#DEFINE TRANSFORMATION
#transform = osr.CoordinateTransformation(vnspatref,geowgsspatref) #Projected VN2000 - Geographic WGS84
transform = osr.CoordinateTransformation(vnspatref,wgsspatref) #Projected Vn2000 - UTM WGS84
#transform = osr.CoordinateTransformation(geovnspatref,geowgsspatref) #Geographic VN2000 - Geographic WGS84
#transform = osr.CoordinateTransformation(geovnspatref,wgsspatref) #Geographic VN2000 - UTM WGS84


In [8]:
#DEFINE OUTPUT REPROJECTED SHAPEFILE
#newfulname = inshpdir.split('.')[-2] + '_GEOWGS.shp' #Output with Geographic WGS84 coordinate system
newfulname = inshpdir.split('.')[-2] + '_UTMWGS.shp' #Output with UTM WGS84 coordinate system
newname = inshpdir.split('.')[-2].split('\\')[-1]
driver = ogr.GetDriverByName('ESRI Shapefile')
outdataset = driver.CreateDataSource(newfulname)
outLayer = outdataset.CreateLayer(newfulname.split('.')[-2].split('\\')[-1],geom_type = ogr.wkbPolygon) #Polygon type 

In [9]:
#GET FIELD DEFINITION FOR OUTPUT SHAPEFILE
inLayerDefn = layer.GetLayerDefn()    
for i in range(0, inLayerDefn.GetFieldCount()):
    fieldDefn = inLayerDefn.GetFieldDefn(i)    
    outLayer.CreateField(fieldDefn)
outLayerDefn = outLayer.GetLayerDefn()

In [10]:
#REPROJECT SHAPEFILES
for feature in layer:
    geom = feature.GetGeometryRef()        
    geom.Transform(transform)    
    outFeature = ogr.Feature(outLayerDefn)    
    outFeature.SetGeometry(geom)        
    for i in range(0,outLayerDefn.GetFieldCount()):            
        outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), feature.GetField(i))    
    outLayer.CreateFeature(outFeature)    
    outFeature = None          
wgsspatref.MorphToESRI()

0

In [11]:
#CLOSE SHAPEFILE
dataset.Destroy() 
outdataset.Destroy()

In [12]:
#CREATE PROJECTION FILE (.PRJ)
newfulnamenoext = newfulname.split('.')[-2]
file = open(newfulnamenoext+'.prj', 'w') 
file.write(wgsspatref.ExportToWkt())
file.close()

In [13]:
#LOAD SHAPEFILES ONTO WEBMAP
#CONVERT SHAPEFILES TO GEOJSON
geopd = gpd.GeoDataFrame.from_file(newfulname)
geojs = geopd.to_crs(epsg='4326').to_json()
#LOAD GEOJSON DATA ONTO WEBMAP
map = folium.Map(zoom_start=7.5)
feature = folium.features.GeoJson(geojs)
map.add_child(feature)
map